## POS_CASH_balance

In [19]:
import pandas as pd
import numpy as np

import lightgbm as lgb
from sklearn.metrics import roc_auc_score, roc_curve

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
save_files = True

In [21]:
# load cash file
df_cash = pd.read_csv('input/POS_CASH_balance.csv')
df_cash.info()
display(df_cash.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001358 entries, 0 to 10001357
Data columns (total 8 columns):
SK_ID_PREV               int64
SK_ID_CURR               int64
MONTHS_BALANCE           int64
CNT_INSTALMENT           float64
CNT_INSTALMENT_FUTURE    float64
NAME_CONTRACT_STATUS     object
SK_DPD                   int64
SK_DPD_DEF               int64
dtypes: float64(2), int64(5), object(1)
memory usage: 610.4+ MB


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [22]:
df_cash.NAME_CONTRACT_STATUS.replace({'Returned to the store':'Returned',
                                      'Amortized debt':'Amortized',
                                      'XNA':np.nan},
                                     inplace=True)

display(df_cash.NAME_CONTRACT_STATUS.value_counts())

df_cash.sort_values(by=['SK_ID_PREV','MONTHS_BALANCE'],inplace=True)
df_cash.reset_index(drop=True,inplace=True)

df_cash.rename(columns={'MONTHS_BALANCE':'MONTH',
                        'CNT_INSTALMENT':'CNT_INST',
                        'CNT_INSTALMENT_FUTURE':'FUTURE_INST',
                        'NAME_CONTRACT_STATUS':'STATUS',
                        'SK_DPD':'DPD',
                        'SK_DPD_DEF':'DPD_DEF'},
               inplace=True)

id_prev_to_curr = df_cash.groupby('SK_ID_PREV').SK_ID_CURR.first()
df_cash.drop('SK_ID_CURR',axis=1,inplace=True)

df_cash = pd.get_dummies(df_cash)

df_cash.head()

Active       9151119
Completed     744883
Signed         87260
Demand          7065
Returned        5461
Approved        4917
Amortized        636
Canceled          15
Name: NAME_CONTRACT_STATUS, dtype: int64

,SK_ID_PREV,MONTH,CNT_INST,FUTURE_INST,DPD,DPD_DEF,STATUS_Active,STATUS_Amortized,STATUS_Approved,STATUS_Canceled,STATUS_Completed,STATUS_Demand,STATUS_Returned,STATUS_Signed
0,1000001,-10,12.0,12.0,0,0,1,0,0,0,0,0,0,0
1,1000001,-9,12.0,11.0,0,0,1,0,0,0,0,0,0,0
2,1000001,-8,2.0,0.0,0,0,0,0,0,0,1,0,0,0
3,1000002,-54,4.0,4.0,0,0,1,0,0,0,0,0,0,0
4,1000002,-53,6.0,3.0,0,0,1,0,0,0,0,0,0,0


In [18]:
agg_funcs = {
    'MONTH':['min','max']
}

,SK_ID_PREV,MONTH,CNT_INST,FUTURE_INST,DPD,DPD_DEF,STATUS_Active,STATUS_Amortized,STATUS_Approved,STATUS_Canceled,STATUS_Completed,STATUS_Demand,STATUS_Returned,STATUS_Signed
0,1000001,-10,12.0,12.0,0,0,1,0,0,0,0,0,0,0
1,1000001,-9,12.0,11.0,0,0,1,0,0,0,0,0,0,0
2,1000001,-8,2.0,0.0,0,0,0,0,0,0,1,0,0,0
3,1000002,-54,4.0,4.0,0,0,1,0,0,0,0,0,0,0
4,1000002,-53,6.0,3.0,0,0,1,0,0,0,0,0,0,0


In [ ]:
df_cash['DAY_DIFF'] = df_cash['DAYS_INSTALMENT']-df_cash['DAYS_ENTRY_PAYMENT']
df_cash['PAY_DIFF'] = df_cash['AMT_INSTALMENT']-df_cash['AMT_PAYMENT']

df_cash.rename(columns={'NUM_INSTALMENT_VERSION':'VERSION',
                        'NUM_INSTALMENT_NUMBER':'NUM',
                        'DAYS_INSTALMENT':'DAY_DUE',
                        'DAYS_ENTRY_PAYMENT':'DAY_PAY',
                        'AMT_INSTALMENT':'AMT',
                        'AMT_PAYMENT':'PAID'},
               inplace=True)

In [ ]:
df_cash.sort_values(by=['SK_ID_PREV','DAY_DUE'],inplace=True)
df_cash.reset_index(inplace=True,drop=True)

df_cash.head()

In [ ]:
agg_funcs = {
    'VERSION':['nunique'],
    'NUM':['first'],
    'DAY_DUE':['min','max','first'],
    'DAY_PAY':['min','max','first'],
    'AMT':['min','max','first','mean'],
    'PAID':['min','max','first','mean'],
    'DAY_DIFF':['min','max','first','mean'],
    'PAY_DIFF':['min','max','first','mean']
}

cash_grouped = df_cash.groupby('SK_ID_PREV').agg(agg_funcs)

cash_grouped.columns = ['INST_'+'_'.join(x) for x in cash_grouped.columns.values]

del df_cash

cash_grouped.head()

In [ ]:
cash_grouped['SK_ID_CURR'] = id_prev_to_curr

cash_idcurr = cash_grouped.groupby('SK_ID_CURR').agg(['min','max','sum','mean'])
cash_idcurr['INST_CNT'] = id_prev_to_curr.value_counts()

cash_idcurr.columns = ['_'.join(x) for x in cash_idcurr.columns.values]
cash_cols = cash_idcurr.columns

del cash_grouped

cash_idcurr.head()

In [ ]:
print('loading train...')
df_train = pd.read_csv('input/application_train.csv',index_col='SK_ID_CURR')

print('merging train...')
df_train = df_train.merge(cash_idcurr,how='left',left_index=True,right_index=True)

if save_files:
    print('saving train...')
    df_train[cash_cols].to_csv('input/app_train_ALL_cash.csv')
    
    print('loading test...')
    df_test = pd.read_csv('input/application_test.csv',index_col='SK_ID_CURR')

    print('merging test...')
    df_test = df_test.merge(cash_idcurr,how='left',left_index=True,right_index=True)
   
    print('saving test...')
    df_test[cash_cols].to_csv('input/app_test_ALL_cash.csv')
    
del cash_idcurr

display(df_train.head())

print('Done!')

## model

In [ ]:
def plot_roc_curve(y_true, y_pred, color='darkorange',plot_random=True,label='ROC curve'):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    score = roc_auc_score(y_true, y_pred)
    
    plt.plot(fpr, tpr, color='darkorange',
             lw=2, label=label+' (area = %0.2f)' % score)
    
    if plot_random:
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
target_counts = df_train.TARGET.value_counts()
targ1_weight = target_counts[0]/target_counts[1]
weights = df_train.TARGET.replace({0:1, 1:targ1_weight}).values

dtypes = df_train.dtypes
cols_categ = dtypes[dtypes=='object'].index.tolist()
for col in cols_categ:
    df_train[col] = df_train[col].astype('category')
    
y_true = df_train.TARGET
X = df_train.drop('TARGET',axis=1)
train_data = lgb.Dataset(X, label=y_true, weight=weights)

    
# train
param = {'objective':'binary', 'metric':'auc'}
bst = lgb.train(param, train_data)

#plot roc curve
plt.figure()
plot_roc_curve(y_true, bst.predict(X))

In [ ]:
plt.figure(figsize=(10,30))

feat_imp = pd.DataFrame({'split':bst.feature_importance(importance_type='split'),
                         'gain':bst.feature_importance(importance_type='gain')},
                        index=bst.feature_name())

total_gain = feat_imp['gain'].sum()
frac_gain = feat_imp['gain']/total_gain
frac_gain = frac_gain.sort_values(ascending=False)

display(frac_gain.head(30))

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(np.arange(len(frac_gain))+1, frac_gain)
plt.xlabel('n_features')
plt.ylabel('frac gain')

plt.subplot(1,2,2)
plt.plot(np.arange(len(frac_gain))+1,frac_gain.cumsum())
plt.xlabel('n_features')
plt.ylabel('cumulative frac gain')

In [ ]:
# select top n columns from cc by importance
n_cols = 60
select_cols = frac_gain[frac_gain.index.str.contains('INST_')].index[:n_cols]

if save_files:
    df_train[select_cols].to_csv('input/app_train_SELECT_cash.csv')
    df_test[select_cols].to_csv('input/app_test_SELECT_cash.csv')
    
frac_gain[select_cols]